<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют
Ответы тоже

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [11]:
import tmdbsimple as tmdb
from tqdm import trange
import numpy as np

In [4]:
tmdb.API_KEY = "82d15a009321e054584df26bf7530dc7"

movies = {}

for i in trange(1000):
    try:
        movies[i] = tmdb.Movies(i).info()['overview']
    except Exception:
        continue

100%|██████████| 1000/1000 [10:01<00:00,  1.66it/s]


In [5]:
movies[99116] = tmdb.Movies(99116).info()['overview']

In [36]:
movies[99114] = tmdb.Movies(68718).info()['overview']

In [49]:
movies[778]

'Monsieur Hulot, Jacques Tati’s endearing clown, takes a holiday at a seaside resort, where his presence provokes one catastrophe after another. Tati’s masterpiece of gentle slapstick is a series of effortlessly well-choreographed sight gags involving dogs, boats, and firecrackers; it was the first entry in the Hulot series and the film that launched its maker to international stardom.'

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1, 2)) 
count_vectorizer_1 = CountVectorizer(ngram_range=(1, 1))

In [51]:
sparse_feature_matrix = count_vectorizer.fit_transform(list(movies.values()))
sparse_feature_matrix

<828x36674 sparse matrix of type '<class 'numpy.int64'>'
	with 71410 stored elements in Compressed Sparse Row format>

In [52]:
sparse_feature_matrix_1 = count_vectorizer_1.fit_transform(list(movies.values()))
sparse_feature_matrix_1

<828x8218 sparse matrix of type '<class 'numpy.int64'>'
	with 32883 stored elements in Compressed Sparse Row format>

In [53]:
index_to_id = np.unique(np.sort(list(movies.keys())))
id_to_index = {}

for index, value in enumerate(index_to_id):
    id_to_index[int(value)] = index

assert len(index_to_id) == len(movies)

In [54]:
def tf_idf(t, d, D):
    
    tf = 0.5 + 0.5*(sparse_feature_matrix_1[id_to_index[d], t]/
                    max(sparse_feature_matrix_1[id_to_index[d], :].data))
    idf = np.log(len(D)/len(sparse_feature_matrix_1[:, t].indices))
    
    return tf * idf

In [55]:
data_tf_idf = []
row_ind = []
col_ind = []
for d in index_to_id:
    for t in sparse_feature_matrix_1[id_to_index[d], :].indices:
        data_tf_idf.append(tf_idf(t, d, index_to_id))
        row_ind.append(id_to_index[d])
        col_ind.append(t)

In [56]:
from scipy.sparse import csr_matrix

tf_idf_sparse_matrix = csr_matrix((np.array(data_tf_idf), (np.array(row_ind), np.array(col_ind))),
                                  shape=np.shape(sparse_feature_matrix))

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

In [57]:
num_2_words_bigrams = {
    v: k
    for k, v in count_vectorizer.vocabulary_.items()
}
num_2_words = {
    v: k
    for k, v in count_vectorizer_1.vocabulary_.items()
}

In [86]:
# There is no film with id 99114 on tmdb, so i have chosen the nearest one (id 99116)
for i in sparse_feature_matrix[id_to_index[99114], :].indices:
    print(num_2_words_bigrams[i], sparse_feature_matrix[id_to_index[99114], i])

in 2004 1
released in 1
and released 1
vaallidharu and 1
as vaallidharu 1
telugu as 1
in telugu 1
rangeela and 1
mr rangeela 1
as mr 1
hindi as 1
into hindi 1
dubbed into 1
later dubbed 1
was later 1
superhit at 1
declared superhit 1
was declared 1
2002 and 1
november 2002 1
29 november 1
on 29 1
released on 1
panneerselvam released 1
by panneerselvam 1
handled by 1
cinematography handled 1
and cinematography 1
raja and 1
shankar raja 1
yuvan shankar 1
by yuvan 1
scored by 1
music scored 1
had music 1
which had 1
film which 1
cast the 1
supporting cast 1
the supporting 1
karunas among 1
and karunas 1
devan and 1
prabhu devan 1
venkat prabhu 1
jayaraman venkat 1
gayatri jayaraman 1
with gayatri 1
roles with 1
lead roles 1
the lead 1
sneha in 1
and sneha 1
srikanth and 1
starring srikanth 1
stanley starring 1
newcomer stanley 1
by newcomer 1
and directed 1
written and 1
film written 1
drama film 1
romantic drama 1
tamil romantic 1
indian tamil 1
2002 indian 1
is 2002 1
april is 1
of apri

In [85]:
for i in tf_idf_sparse_matrix[id_to_index[99114], :].indices:
    print(num_2_words[i], tf_idf_sparse_matrix[id_to_index[99114], i])

2002 3.87377098317
2004 3.21165763755
29 3.83943608822
among 2.65118377869
and 0.259108700008
april 3.61311484225
as 0.897430612659
at 1.1586658698
box 2.91975728111
by 0.973162056938
cast 2.52367317794
cinematography 3.83943608822
declared 3.44335198504
devan 3.83943608822
directed 2.91975728111
drama 2.18779508371
dubbed 3.04726788187
english 2.4194894312
film 1.25440608866
gayatri 3.83943608822
had 2.18779508371
handled 3.83943608822
hindi 3.44335198504
in 0.430634617115
indian 2.81557353438
into 1.0803996669
is 0.408643478445
jayaraman 3.83943608822
karunas 3.83943608822
later 2.29197883045
lead 2.3737507411
maadhathil 3.83943608822
month 3.44335198504
mr 2.72748743162
music 2.52367317794
newcomer 3.44335198504
november 3.83943608822
of 0.165310672198
office 2.72748743162
on 0.695859967909
others 2.72748743162
panneerselvam 3.83943608822
prabhu 3.83943608822
raja 3.83943608822
rangeela 3.83943608822
released 3.4281763671
roles 3.44335198504
romantic 2.52367317794
scored 3.839436088

## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [20]:
import pandas as pd

ratings = pd.read_csv("ml-20m/ratings.csv")

In [21]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [24]:
index_to_item_id_mapping = [x for x in ratings["movieId"].sort_values().unique()]
item_id_to_index_mapping = {}
for index, value in enumerate(index_to_item_id_mapping, start=1):
    item_id_to_index_mapping[int(value)] = index

assert len(index_to_item_id_mapping) == 26744

In [29]:
#ratings = pd.DataFrame.from_records(
#    {"userId": row.userId, "movieId": item_id_to_index_mapping[int(row.movieId)], "rating": row.rating}
#    for row in ratings.itertuples())

In [30]:
ratings.head()

,movieId,rating,userId
0,2,3.5,1
1,29,3.5,1
2,32,3.5,1
3,47,3.5,1
4,50,3.5,1


In [31]:
links = pd.read_csv("ml-20m/links.csv")

In [35]:
links[links['movieId']==99114]

,movieId,imdbId,tmdbId
20124,99114,1853728,68718.0


In [83]:
def run_cv(predictor_class, fold_count=5, top_size=10):
    users = list(range(len(ratings["userId"].unique())))
    fold_size = len(users) // fold_count

    results = []

    for fold_start in range(fold_count):
        fold = users[fold_start:fold_start+fold_size]

        ratings["random"] = np.random.random(size=(ratings.shape[0], 1))
        idx = ratings.sort_values(by="random") \
            .reset_index() \
            .groupby("userId") \
            .last()["index"]

        mask = ratings.index.isin(idx) & ratings["userId"].isin(fold)
        train_df = ratings[~mask]
        test_df = ratings[mask]

        predictor = predictor_class(train_df, test_df)
        
        sum_ = 0.0
        for row in test_df.itertuples():
            prediction = predictor.predict(row)
            if prediction is None:
                continue
                
            sum_ += float(row.movieId in prediction)

        results.append(sum_ / test_df.shape[0])
    
    return np.mean(results)

In [61]:
df_movie = pd.read_csv("ml-20m/movies.csv")

In [62]:
from collections import Counter

genres = []

for row in df_movie.itertuples():
    genre_list = row.genres.split("|")
    genres.extend(genre_list)

cntr = Counter(genres)

In [63]:
cntr

Counter({'(no genres listed)': 246,
         'Action': 3520,
         'Adventure': 2329,
         'Animation': 1027,
         'Children': 1139,
         'Comedy': 8374,
         'Crime': 2939,
         'Documentary': 2471,
         'Drama': 13344,
         'Fantasy': 1412,
         'Film-Noir': 330,
         'Horror': 2611,
         'IMAX': 196,
         'Musical': 1036,
         'Mystery': 1514,
         'Romance': 4127,
         'Sci-Fi': 1743,
         'Thriller': 4178,
         'War': 1194,
         'Western': 676})

In [64]:
cntr.most_common(5)

[('Drama', 13344),
 ('Comedy', 8374),
 ('Thriller', 4178),
 ('Romance', 4127),
 ('Action', 3520)]

In [66]:
df_joined = pd.merge(ratings, df_movie[["movieId", "genres"]], on="movieId", how="inner")

In [67]:
def genre_list(x):
    return list(set("|".join(x.tolist()).split("|")))

def genre_list_len(x):
    return len(genre_list(x))

def popular_genres(x):
    return Counter("|".join(x.tolist()).split("|")).most_common(3)

df_users = df_joined.groupby('userId').agg({'genres': [genre_list, genre_list_len, popular_genres]})

In [68]:
df_users.head()

genres                 \
                                               genre_list genre_list_len   
userId                                                                     
1       [Sci-Fi, Fantasy, Action, Film-Noir, Romance, ...             19   
2       [Action, Fantasy, Sci-Fi, Film-Noir, Romance, ...             17   
3       [Sci-Fi, Fantasy, Action, Film-Noir, Thriller,...             18   
4       [Sci-Fi, Action, Fantasy, Film-Noir, Romance, ...             16   
5       [Action, Fantasy, Sci-Fi, Film-Noir, Romance, ...             17   

                                                     
                                     popular_genres  
userId                                               
1        [(Drama, 77), (Comedy, 49), (Romance, 31)]  
2        [(Drama, 34), (Comedy, 21), (Romance, 13)]  
3       [(Drama, 94), (Comedy, 55), (Thriller, 40)]  
4           [(Drama, 13), (Comedy, 8), (Action, 8)]  
5        [(Drama, 39), (Comedy, 25), (Romance, 16)]

In [69]:
df_joined[df_joined["userId"] == 1].head()

,movieId,rating,userId,genres
0,2,3.5,1,Adventure|Children|Fantasy
22243,29,3.5,1,Adventure|Drama|Fantasy|Mystery|Sci-Fi
30763,32,3.5,1,Mystery|Sci-Fi|Thriller
75743,47,3.5,1,Mystery|Thriller
118992,50,3.5,1,Crime|Mystery|Thriller


In [71]:
df_counts = df_joined.groupby("movieId").count().reset_index()
df_counts["count"] = df_counts["rating"]
for col in ("userId", "genres", "rating"):
    del df_counts[col]

In [72]:
result = pd.merge(df_joined, df_counts, on="movieId")

In [73]:
result.head()

,movieId,rating,userId,genres,count
0,2,3.5,1,Adventure|Children|Fantasy,22243
1,2,3.0,5,Adventure|Children|Fantasy,22243
2,2,3.0,13,Adventure|Children|Fantasy,22243
3,2,3.0,29,Adventure|Children|Fantasy,22243
4,2,3.0,34,Adventure|Children|Fantasy,22243


In [75]:
from collections import defaultdict

genre_to_movies = defaultdict(set)

for row in result.itertuples():
    genres = row.genres.split("|")
    for genre in genres:
        genre_to_movies[genre].add((row.movieId, row.count))

In [76]:
genre_to_top_movies = {}

for k, v in genre_to_movies.items():
    genre_to_top_movies[k] = sorted(list(v), key=lambda p: -p[1])[:10]

In [77]:
df_users_new = df_users.reset_index()

In [84]:
class ContentBasedPredictor(object):
    def __init__(self, train_df, test_df):
        pass
    
    def predict(self, row):
        top = list(df_users_new[df_users_new["userId"] == row.userId][("genres", "popular_genres")])
        
        if not top:
            return
        
        top_genres = [k for k, _ in top[0]]
        recommendations = []
        
        i_to_size = {0: 5, 1: 3, 2: 2}
        for i, genre in enumerate(top_genres):
            for movie, _ in genre_to_top_movies[genre][:i_to_size[i]]:
                recommendations.append(movie)
        
        return recommendations
    
run_cv(ContentBasedPredictor)

0.043967382109264867